In [2]:
import csv
import logging
import requests
from bs4 import BeautifulSoup
import time
import re
import json

In [3]:
logging.basicConfig(format='%(message)s')

In [4]:
def normalize_query(q):
    q = q.replace('\n', '')
    q = q.replace('（', '(')
    q = q.replace('）', ')')
    q = re.sub(r"\(.+\)$", "", q)
    q = re.sub('(!|\u3000|/|\\s|>|<|\\.)+', " ", q)
    return q

In [5]:
def search(q):
    url_search = 'https://eiga.com/search/{}' .format(requests.utils.quote(normalize_query(q), safe=''))
    res_search = requests.get(url_search )
    res_search.encoding = res_search.apparent_encoding
    
    soup_search = BeautifulSoup(res_search.content, "lxml")
    result =  soup_search.find('section', attrs={"id": "rslt-movie"})
    if result != None:
        path = result.find('li', attrs={"class": "col-s-3"}).find('a')["href"]
        return path
    else:
        return None

In [6]:
def scrape(query, movie_id, year):
    data = {
        "id": -1,
        "rating": -1,
        "check-in":-1,
        "review-count":-1
    }
    print("START : " + query)
    path = search(query)
    
    if path is None:
        logging.warning("**************************************************")
        logging.warning(query + " HAS NO RESULT")
        logging.warning("**************************************************")
        with open('./eigacom_id_table_{}.txt'.format(year), 'a') as f:
            print(str(movie_id), "-1")
            f.write(str(movie_id) + '\t' + '-1' + '\n')
        return None
    
    url_review = 'https://eiga.com' + path
    eigacom_id = re.sub("\\D", "", path)
    
    with open('./eigacom_id_table_{}.txt'.format(year), 'a') as f:
        print(str(movie_id), eigacom_id)
        f.write(str(movie_id) + '\t' + eigacom_id + '\n')
    
    res = requests.get(url_review)
    res.encoding = res.apparent_encoding
    soup = BeautifulSoup(res.content, "lxml")
    
    rating = soup.find('span', attrs={"class": "rating-star"})
    review_count = soup.find('span', attrs={"itemprop": "reviewCount"})
    check_in = soup.find('a', attrs={"class": "icon-movie-checkin"}).find('strong')
    data["rating"] = 0 if rating.text == '－' else float(rating.text)
    data["review-count"] = 0 if review_count is None else int(review_count.text)
    data["check-in"] = 0 if check_in is None else int(check_in.text)
        
    return data

In [10]:
def main():
#     years = ['2017', '2016', '2015', '2014', '2013']
    years = ['2014', '2013']
    for y in years:
        print(y)
        with open( '../{}_movie_clean'.format(y), 'r') as movie_clean:
            for line in csv.reader(movie_clean, delimiter='\t'):
                movie_id, title, *_ = line
                if y == "2014" and int(movie_id) <= 540: #あとで消す
                    continue
                    
                output_file = './{0}/{1}.json'.format(y, movie_id)
                with open(output_file, 'w') as f:
                    print(movie_id)
                    data = scrape(title,movie_id, y)
                    if data == None:
                        continue
                    data["id"] = int(movie_id)
                    json.dump(data, f, ensure_ascii=False, indent=2) 
                time.sleep(1)

In [11]:
if __name__ == '__main__':
    main()

2014
541
START : 劇場版 カードファイト!! ヴァンガード 3つのゲーム
541 80355
542
START : フランシス・ハ
542 79705
543
START : リスボンに誘われて
543 79780
544
START : ハニー・フラッパーズ
544 79587
545
START : サスペクト 哀しき容疑者
545 80459
546
START : ガーディアンズ・オブ・ギャラクシー
546 84133
547
START : Nas/タイム・イズ・イルマティック
547 80575
548
START : 悪魔は誰だ
548 80598
549
START : 銀座並木通り クラブアンダルシア
549 80632
550
START : さまよう小指
550 80673
551
START : 猿の惑星：新世紀（ライジング）
551 78803
552
START : ぼんとリンちゃん
552 80238
553
START : 柘榴坂の仇討
553 79913
554
START : カラアゲ★USA
554 80393
555
START : ウィークエンドはパリで
555 79781
556
START : 劇場版 稲川怪談 かたりべ
556 80552
557
START : バツイチは恋のはじまり
557 79159
558
START : がじまる食堂の恋
558 80184
559
START : NY心霊捜査官
559 80543
560
START : ゲッタウェイ スーパースネーク
560 77931
561
START : フランキー&アリス
561 80591
562
START : 劇場版 幼獣マメシバ 望郷篇
562 80486
563
START : ピーター・ブルックの世界一受けたいお稽古
563 80195
564
START : トリハダ -劇場版2-
564 80576
565
START : ぼくらは動物探検隊〜富士サファリパークで大冒険
565 80654
566
START : 劇場版 零〜ゼロ〜
566 80311
567
START : マザー
567 87431
568
START : 幻肢
568 79664
569
START : アンナプルナ南壁 7,400mの男たち

**************************************************
アンナプルナ南壁 7,400mの男たち HAS NO RESULT
**************************************************


569 -1
570
START : アルゲリッチ 私こそ、音楽！
570 80322
571
START : アバウト・タイム 〜愛おしい時間について〜
571 79496
572
START : 劇場版 gdgd妖精s（ぐだぐだフェアリーズ） っていう映画はどうかな…？
572 80363
573
START : 少女は異世界で戦った
573 80048
574
START : ジャージー・ボーイズ
574 79806
575
START : 不機嫌なママにメルシィ!
575 80067
576
START : 聖者たちの食卓
576 77589
577
START : 記憶探偵と鍵のかかった少女
577 79804
578
START : 世界一美しいボルドーの秘密
578 79783
579
START : 荒野の千鳥足
579 80553
580
START : ファーナス/訣別の朝
580 79617
581
START : ジェラシー
581 80586
582
START : 愛の棘
582 80628
583
START : ハンガー・ゼット
583 80715
584
START : アキラNo.2
584 80307
585
START : 風邪(ふうじゃ)
585 80960
586
START : 悪童日記
586 79643
587
START : レッド・ファミリー
587 79294
588
START : 蜩ノ記（ひぐらしのき）
588 78319
589
START : 太陽の坐る場所
589 79724
590
START : FRANK -フランク-
590 80487
591
START : ミリオンダラー・アーム
591 80380
592
START : レクイエム 最後の銃弾
592 80627
593
START : サル ウインドウ・ピリオド
593 80943
594
START : アンダー・ザ・スキン 種の捕食
594 57779
595
START : 小川町セレナーデ
595 80394
596
START : サミーとシェリー 七つの海の大冒険
596 80667
597
START : LAST LOVE/愛人
597 80701
598
START : カバディーン!!!!!!!〜嗚呼・花吹雪高校篇

**************************************************
弾丸刑事(デカ) 怒りの奪還 HAS NO RESULT
**************************************************


654 -1
655
START : 茜色クラリネット
655 80999
656
START : サボタージュ
656 80167
657
START : 嗤う分身
657 79299
658
START : 花宵道中
658 80041
659
START : 福福荘の福ちゃん
659 79359
660
START : トワイライト ささらさや
660 80310
661
START : 最後の命
661 80514
662
START : 100歳の華麗なる冒険（ スウェーデン）
662 80614
663
START : 0.5ミリ


...

477 -1
478
START : ブッダ・マウンテン 希望と祈りの旅
478 78792
479
START : ラスト・シャンハイ
479 79019
480
START : パッション
480 86876
481
START : ルノワール 陽だまりの裸婦
481 78050
482
START : レッド・ドーン
482 77850
483
START : フローズン・グラウンド
483 78592
484
START : ランナウェイ 逃亡者
484 78046
485
START : ムード・インディゴ うたかたの日々
485 78602
486
START : R100
486 78690
487
START : 飛べ!ダコタ
487 78881
488
START : ゲキ×シネ シレンとラギ
488 78747
489
START : 今日子と修一の場合
489 78744
490
START : バトル・オブ・ヒロミくん! The High School SAMURAI BOY
490 79006
491
START : トランス
491 91028
492
START : 死霊館
492 88416
493
START : ファントム/開戦前夜
493 78216
494
START : 天使の処刑人 バイオレット&デイジー
494 78059
495
START : カイロ・タイム 異邦人
495 78919
496
START : 黒い殺意
496 79052
497
START : おしん
497 78028
498
START : 陽だまりの彼女
498 77883
499
START : キッズ・リターン 再会の時
499 78977
500
START : 男子高校生の日常
500 78605
501
START : 埼玉家族
501 79102
502
START : 東京シャッターガール
502 79145
503
START : ダンシング・クィーン
503 79166
504
START : フィスト・オブ・レジェンド
504 79164
505
START : ミリオネア・オン・ザ・ラン
505 79165
506
START : ダイアナ
506 78308
507
START : ゴースト・エージェント R.I.P.

**************************************************
ガチバン エクスペンデッド HAS NO RESULT
**************************************************


516 -1
517
START : 101回目のプロポーズ SAY YES
517 78784
518
START : 殺人漫画
518 79113
519
START : グランド・イリュージョン
519 84434
520
START : ナゾトキネマ マダム・マーマレードの異常な謎 出題編
520 78873
521
START : デッドマン・ダウン
521 78318
522
START : セイフ ヘイヴン
522 78226
523
START : アフターショック
523 78812
524
START : ロスト・ボディ
524 78811
525
START : 僕が星になるまえに
525 78962
526
START : バーバリアン怪奇映画特殊音響効果製作所
526 78810
527
START : ZMフォース ゾンビ虐殺部隊
527 79112
528
START : ハンナ・アーレント
528 77521
529
START : 道化死てるぜ!
529 78813
530
START : 潔く柔く
530 77839
531
START : 彌勒 MIROKU
531 77163
532
START : 女たちの都 ワッゲンオッゲン
532 57610
533
START : 自分の事ばかりで情けなくなるよ
533 78998
534
START : 花様 たゆたう想い ( 台湾)
534 79000
535
START : マッキー
535 78718
536
START : コールド・ウォー 香港警察 二つの正義
536 78600
537
START : 恐怖ノ黒洋館
537 79091
538
START : スティーブ・ジョブズ
538 83404
539
START : 42 世界を変えた男
539 58340
540
START : 2ガンズ
540 58262
541
START : パーシー・ジャクソンとオリンポスの神々/魔の海
541 57680
542
START : 恋するリベラーチェ
542 79075
543
START : アメリカン・パイパイパイ! 完結編 俺たちの同騒会
543 58115
544
START : ある愛へと続く旅
544 78057
545
START : 劇場版 SPEC〜結〜

**************************************************
バックコーラスの歌姫(ディーバ)たち HAS NO RESULT
**************************************************


633 -1
634
START : 消えたシモン・ヴェルネール
634 79483
635
START : 少女は自転車にのって
635 79133
636
START : 自由と壁とヒップホップ
636 79240
637
START : 楊家将〜烈士七兄弟の伝説〜
637 79234
638
START : FLU 運命の36時間
638 79404
639
START : ヴァイキングダム
639 79539
640
START : ゆめのかよいじ
640 77662
641
START : 武士の献立
641 78136
642
START : カノジョは嘘を愛しすぎてる
642 58057
643
START : 楽隊のうさぎ
643 77757
644
START : 祭の馬
644 79162
645
START : 始まりも終わりもない
645 78335
646
START : 映画 かいけつゾロリ まもるぜ!きょうりゅうのたまご
646 78912
647
START : ウォーキングwithダイナソー
647 57679
648
START : オンリー・ラヴァーズ・レフト・アライヴ
648 79215
649
START : プレーンズ
649 81085
650
START : ウォールストリート・ダウン
650 79540
651
START : キューティー&ボクサー
651 79043
652
START : フォンターナ広場 イタリアの陰謀
652 78557
653
START : マンボーグ
653 79427
654
START : ファイアbyルブタン
654 78311
655
START : パリ、ただよう花
655 78022
656
START : ルートヴィヒ
656 50747
657
START : 永遠の0
657 58291
658
START : 麦子さんと
658 78246
659
START : 赤々煉恋
659 79201
660
START : ハンガー・ゲーム2
660 78219
661
START : くもりときどきミートボール2 フード・アニマル誕生の秘密
661 78317
662
START : 皇帝と公爵
662 78040
